# ID2214/FID3214 Assignment 2 Group no. [2]
### Project members: 
[Abyel Tesfay, Abyel@kth.se]
[Farrokh Bolandi, Farrokh@kth.se]
[Ezio Christofoli, Ezioc@kth.se]
[Enter Name, email]

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [1]:
import numpy as np
import pandas as pd
import time
from numpy.linalg import norm #its part of numpy


## Reused functions from Assignment 1

In [2]:
def create_column_filter(df):
    new_df = df.copy()
    new_df = new_df.filter([e for e in new_df.columns if new_df[e].nunique() > 1 or e in ['CLASS', 'ID']], axis=1)
    return new_df, list(new_df.columns)

def create_normalization(df, normalizationtype="minmax"):
    new_df = df.copy()
    normalization = {}
    for e in new_df.columns:
        if e not in ['CLASS', 'ID'] and new_df[e].dtypes in ["float64", "int64"]:
            if normalizationtype == "minmax":
                min = new_df[e].min()
                max = new_df[e].max()
                new_df[e] = [(x-min)/(max-min) for x in new_df[e]]
                normalization[e] = ("minmax", min, max)
            elif normalizationtype == "zscore":    
                mean = new_df[e].mean()
                std = new_df[e].std()
                new_df[e] =  [((x-mean)/std) for x in new_df[e]]
                normalization[e] = ("zscore", mean, std)
    return new_df, normalization

def create_imputation(df):
    new_df = df.copy()
    imputation = {}
    for e in new_df.columns:
        # numerical values
        if e not in ['CLASS', 'ID'] and new_df[e].dtypes in ["float64", "int64"]:
            if new_df[e].nunique() < 1:
                new_df[e].fillna(0, inplace=True)
            new_df[e].fillna(new_df[e].mean(), inplace=True)
            imputation[e] = new_df[e].mean()
        #categorical or object values
        else:
            #if it is an object
            if new_df[e].dtypes == "object":
                new_df[e].astype('category')
            if new_df[e].nunique() < 1:
                new_df[e].fillna(new_df[e][0], inplace=True)
            new_df[e].fillna(new_df[e].mode()[0], inplace=True)
            imputation[e] = new_df[e].mode()[0]
    return new_df, imputation


def create_one_hot(df):
    new_df = df.copy()
    handle = new_df.filter([e for e in new_df.columns if new_df[e].dtype.name in ['object', 'category'] or e not in ['CLASS', 'ID']], axis=1)
    one_hot = {}
    for e in handle.columns:
            features = np.sort(handle[e].unique())                                      #3
            for i in features:  
                new_df[e + "-" + str(i)] = [1.0 if x == i else 0.0 for x in handle[e]]
                new_df[e + "-" + str(i)].astype('float')                                     #4
            one_hot[e] = features
            new_df.drop(e, axis=1, inplace=True)                                        #5
    return new_df, one_hot

def apply_column_filter(df, column_filter):
    new_df = df.copy()
    new_df = new_df.filter(items=column_filter, axis=1)  
    return new_df

def apply_normalization(df, normalization):
    new_df = df.copy()
    for e in new_df.columns:
        if e in normalization:
            a = normalization.get(e)[1]
            b = normalization.get(e)[2]
            new_df[e] = [(x-a)/(b-a) for x in new_df[e]] # works with minmax or zscore
    return new_df

def apply_imputation(df, imputation):
    new_df = df.copy()
    new_df.fillna(imputation, inplace=True)
    return new_df

def apply_one_hot(df, one_hot):
    new_df = df.copy()
    for e in new_df.columns:
        if e in one_hot and e not in ['CLASS', 'ID']:
            for i in one_hot[e]: 
                new_df[e + "-" + str(i)] = [1.0 if x == i else 0.0 for x in new_df[e]]
                new_df[e + "-" + str(i)].astype('float')                                          #4
            new_df.drop(e, axis=1, inplace=True)                                                  #5
    return new_df

def accuracy(df, correctlabels):
    truCases, allCases = 0, 0
    for r in df.index:
        if correctlabels[r] == df.loc[r][:].idxmax(axis=0):
            truCases += df.loc[r][:].max(axis=0)
            display(correctlabels[r] + " is EQUAL ")
            display(df.loc[r][:].idxmax(axis=0))
        else:
            display(correctlabels[r] + " is not equal ")
            display(df.loc[r][:].idxmax(axis=0))
        allCases += df.loc[r][:].max(axis=0)
    return truCases/allCases

def brier_score(df, correctlabels):
    squared_sum = 0
    row = 0
    for label in correctlabels:
        i = np.where(df.columns==label)[0]
        for col in df.columns:
            squared_sum += (1 - df.loc[row, label])**2 if label==col else df.loc[row, col]**2
        row+=1
    return squared_sum/df.index.size

def auc(df, correctlabels):
    auc=0
    
    for col in df.columns:
        df2 = pd.concat([df[col], pd.Series(correctlabels, name='correct')], axis=1)
        # get dummies for correct labels and sort descending
        df2 = pd.get_dummies(df2.sort_values(col, ascending=False))
        
        # move col to first for easier total tp and fp calculation
        tmp=df2.pop('correct_' + str(col))
        # get the col frequency for calculating weighted AUCs
        col_frequency=tmp.sum()/tmp.index.size
        df2.insert(1, tmp.name, tmp)
#         display(df2)
        scores={}
        # populate the scores dictionary for column i.e. key=score, value=[tp_sum, fp_sum]
        for row in df.index:
            key=df2.iloc[row, 0]
#             current=np.zeros(score_dimension, dtype=np.uint) if scores.get(key) is None else scores[key]
            current=np.zeros(2, dtype=np.uint) if scores.get(key) is None else scores[key]
            to_add=np.array([1,0]) if df2.iloc[row, 1]==1 else np.array([0,1])
#             scores[key]=current+np.array([df2.iloc[row, 1], 0])
            scores[key]=current+to_add
#         print(scores)

        # calculate auc based on scores
#         print(f'scores={scores}')
        cov_tp=0
        column_auc=0
        tot_tp=0
        tot_fp=0
        # calculate total tp and fp 
        for value in scores.values():
            tot_tp+=int(value[0])
            tot_fp+=int(value[1])
            
        # same algorithm as in the lecture (bad naming though)
        for i in scores.values():
            if i[1] == 0:
                cov_tp+=i[0]
            elif i[0] == 0:
                column_auc += (cov_tp/tot_tp)*(i[1]/tot_fp)
            else:
                column_auc += (cov_tp/tot_tp)*(i[1]/tot_fp)+(i[0]/tot_tp)*(i[1]/tot_fp)/2
                cov_tp += i[0]

#         print(column_auc)    
        
        auc+=col_frequency*column_auc
    
    return auc

## 1. Define the class kNN

In [3]:

""" represents an instace of k nearest neighbours
uses the function fit to train the learning algorithm
predict then uses the train data to predict certain class
label every for test instance 
"""
class kNN:
    def __init__(self):
        self.column_filter = None         # filter for columns with only missing values or 1 unique value
        self.imputation = None            # a mapping from colum to value to impute with
        self.normalization = None         # normalization from column to the max/min values to normalise with
        self.one_hot = None               # a mapping from a column to all possible categorical features
        self.labels = None                # all possible class labels from the training data
        self.training_labels = None       # a series containing class labels 
        self.training_data = None         # the training data used for predicting test instances
        self.training_time = None         # the time to train the algorithm
        self.df_Before = None             # USED FOR COMPARISON
      
    """takes a dataframe df, a normalization type (default=minmax), 
    prepare the current kNN instance for predictng test values
    """
    def fit(self, df, normalizationtype="minmax"): 
        df, self.column_filter = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.normalization = create_normalization(df, normalizationtype)
        df, self.one_hot = create_one_hot(df)                                  # not needed
        self.training_labels = df["CLASS"].astype('category')
        self.labels = self.training_labels.cat.categories
        self.training_data = df.drop(["ID", "CLASS"], axis=1, errors="ignore").to_numpy()

    """takes a dataframe df, a constant k (default=5),
    returns a dataframe consisting of class labels as columns
    and rows as the probabilities for each class label for each
    row in df
    
    the probability of each label is estimated by the relative frequency in the set of labels
    from the k nearest (Euclidean distance) neighbors in training_data.
    """
    def predict(self, df, k=5):
        
        #1
        df_dropped = df.drop(["ID", "CLASS"], axis=1, errors="ignore")
        df_dropped = apply_column_filter(df_dropped, self.column_filter)
        df_dropped = apply_imputation(df_dropped, self.imputation)
        df_dropped = apply_normalization(df_dropped, self.normalization)
        #df_dropped = apply_one_hot(df_dropped, self.one_hot)                   # not needed
        
        #2
        predictions = pd.DataFrame(index=[], columns=self.labels)
        for i in df_dropped.index: 
            test_set = df_dropped.iloc[i,:].to_numpy()
            freq = self.get_nearest_neighbor_predictions(test_set, k)
            predictions = predictions.append(freq, ignore_index=True)
        return predictions
        
    def get_nearest_neighbor_predictions(self, x_test, k):
        distance_to_labels = sorted([(norm(x_test - x_train), i) for i, x_train in enumerate(self.training_data)])
        k_nearest = [self.training_labels[distance_to_labels[i][1]] for i in range(k)]
        class_freq = {c: k_nearest.count(c)/len(k_nearest) for c in self.labels}
        return class_freq

# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies

In [4]:
glass_train_df = pd.read_csv("glass_train.csv")
glass_test_df = pd.read_csv("glass_test.csv")
temp = kNN()

#fit
t0 = time.perf_counter()
temp.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

#predict
test_labels = glass_test_df["CLASS"]
k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = temp.predict(glass_test_df, k=k_values[i])
    display(predictions)
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i], time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels)]

train_labels = glass_train_df["CLASS"]
predictions = temp.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time: 0.38 s.


ValueError: operands could not be broadcast together with shapes (9,) (573,) 

In [ ]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

In [ ]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [ ]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
class NaiveBayes:
    def __init__(self):
        self.column_filter = None                   # filter for columns with only missing values or 1 unique value
        self.binning = None                         # mapping of discrete intervals e.g (1-3), (3-5) etc.
        self.class_priors = None                    # mapping from class labels (from CLASS column) to the relative frequency of each label in the column
        self.labels = None                          # a list of the different categories from the CLASS column
        self.feature_class_value_counts = None      # mapping from feature to the number training instances with ...
        self.feature_class_counts = None            # mapping from feature to number training instances with ...

    def fit(self, df, nobins=10, bintype="equal-width"):
        # prepare data
        df, self.column_filter = create_column_filter(df)
        df, self.binning = create_bins(df, nobins, bintype)
        self.labels = df["CLASS"].astype('category').cat.categories
        self.class_priors = {c: sum(df["CLASS"] == c)/len(df["CLASS"]) for c in self.labels} # P(H)
        
        # for P(H|C)
        df_no_classId = df.drop(["ID", "CLASS"], axis=1)
        self.feature_class_value_counts = {(f, c, val): sum(df_no_classId[f] == val) for f in df_no_classId.columns for c in self.labels for val in pd.unique(df_no_classId[f])}
        self.feature_class_counts = {(f, c): sum(df["CLASS"] == c) for f in df_no_classId.columns for c in self.labels}

# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of
#                                   value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some
#                                   value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors

# Class priors is P(H)
# P(E|H) is P(e1|H) * P(e2|H) ... 

In [ ]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

In [ ]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

### Comment on assumptions, things that do not work properly, etc.